In [1]:
#1.创建数据库连接--------------------------------------------------------------------

import pymysql

def connect_db():
    try:
        #建立connect路径
        conn = pymysql.connect(host = 'sql.b51.vhostgo.com', db = 'passvisor', user = 'passvisor', password = 'Xudamin123', charset = 'utf8')
        #建立cursor
        cur = conn.cursor()
        
        print('Connection Succeeded!')
        
        return conn, cur    #返回路径、cursor，供下文使用
    except:
        return False       #如上述代码请求不成功，返回False字样
    
#2.在数据库创建相应的表（未必会用到）---------------------------------------------------------------------------------------------

def create_table_predict():
    if not connect_db() == False:
        conn, cur = connect_db()
        sql = """create table predicted_value(
                 id int(10) auto_increment primary key,
                 date date,
                 region varchar(64),
                 accuracy float,
                 day_1 float,
                 day_2 float,
                 day_3 float,
                 day_4 float,
                 day_5 float,
                 day_6 float,
                 day_7 float)"""
        cur.execute(sql)
        
        cur.execute('desc predicted_value')
        print(cur.fetchall())

def create_table_corr():
    if not connect_db() == False:
        conn, cur = connect_db()
        sql = """create table copredict_corr(
                 id int(10) auto_increment primary key,
                 date date,
                 corr_X1 float,
                 corr_X2 float)"""
        cur.execute(sql)
        
        cur.execute('desc copredict_corr')
        print(cur.fetchall())
    
#2.生成日期---------------------------------------------------------------------------

import datetime
import pandas as pd

def create_date(datestart,dateend):

    # 转为日期格式
    datestart=datetime.datetime.strptime(datestart,'%Y-%m-%d')
    dateend=datetime.datetime.strptime(dateend,'%Y-%m-%d')
    date_list = []
    date_list.append(datestart.strftime('%Y-%m-%d'))
    
    while datestart<dateend:
        # 日期叠加一天
        datestart+=datetime.timedelta(days=+1)
        # 日期转字符串存入列表
        date_list.append(datestart.strftime('%Y-%m-%d'))
    
    print('Date Generated!')
    
    return date_list

#3.读取数据表---------------------------------------------------------------------------

import pandas as pd
import pymysql

def read_db_data(table_name):
    if not connect_db() == False:                     #如果连接数据库没有不成功，
        conn, cur = connect_db()                       #connect路径与cursor来自connect_db()
        sql = 'select * from %s'%(table_name)         #sql指令执行全选数据表内容
        db_data = pd.read_sql(sql = sql, con = conn) #用pandas格式读取
    
    df_origin = {}                                   #将读取回来的数据用字典呈现
    
    df_origin['date'] = db_data['date']
    df_origin['price'] = db_data['price']
    
    print('Data Table Obtained!')
    
    return df_origin


#4.创建df表----------------------------------------------------------------------------

import pandas as pd
import datetime

def create_df():
    #读取y、x1、x2数据
    y = read_db_data('copredict_source_pork')
    x1 = read_db_data('copredict_source_maize')
    x2 = read_db_data('copredict_source_stock')
    
    #将y、x1、x2格式转为DataFrame
    y = pd.DataFrame(y)
    x1 = pd.DataFrame(x1)
    x2 = pd.DataFrame(x2)

    #创建基础的df数据表以供数据写入
    df = {}
    df['date'] = create_date('2019-02-19', '2020-02-19')
    df['y'] = [0.00] * len(df['date'])
    df['x1'] = [0.00] * len(df['date'])
    df['x2'] = [0.00] * len(df['date'])
    df = pd.DataFrame(df)

    #将y、x1、x2的数据逐个赋值到df表（如果相应日期没有数据，则保留原来的“0.00”）
    for i in range(len(df['date'])):
        r_drop = False
        _value = datetime.datetime.strptime(df['date'][i], '%Y-%m-%d').date()  #转换df['date']数据的格式为datetime.date
        if _value in y['date'].tolist():                                       #如果df的日期在为其赋值的数据表内的话，
            _index = y.iloc[:, 0].tolist().index(_value)                       #先找到赋值数据表相应日期的索引（index）
            df['y'][i] = y['price'][_index]                                    #将index对应的‘price’赋值到df中
        else:
            r_drop = True

        if _value in x1['date'].tolist():
            _index = x1.iloc[:, 0].tolist().index(_value)
            df['x1'][i] = x1['price'][_index]
        else:
            r_drop = True

        if _value in x2['date'].tolist():
            _index = x2.iloc[:, 0].tolist().index(_value)
            df['x2'][i] = x2['price'][_index]
        else:
            r_drop = True
            
        if r_drop ==True:
            df.drop([i], inplace = True)
            #print('已删除第{}行'.format(i))
    
    df.drop(['date'], axis = 1, inplace = True)
    
    print('Variables for Calcutaion Obtained!')
    
    return df  #返回df数据表

df = create_df()

#5.相关系数---------------------------------------------------------------------------

def corr():

    corr = df.corr()   #Done
    print('Coefficients of Correlation obtained!')
    return corr

corr = corr()
corr

#6.数据预处理---------------------------------------------------------------------------

import numpy as np
from sklearn import preprocessing

def general_data(df, forcast_len):     #定义预处理函数
    #df.fillna(-9999, inplace = True)   #如果某项为空，赋一个极端值在数据表的副本上，不对表造成影响（去除空值）
    df['label'] = df['y'].shift(-forcast_len) #生成y数据原型（y移动forcast_len单位）

    X = np.array(df.drop(['label'], 1)) #生成X数据的数组

    X = preprocessing.scale(X)          #使X数组作正态分布处理
   
    X_lately = X[0:forcast_len, :]      #先生成X_lately的数据，也就是从X数据原型中取出的部分

    X = X[:-forcast_len]                 #生成X数据，去除X_lately的部分（前面X已经数组化X_lately不需要再数组化）
    
    df.dropna(inplace = True)           #df（y和X的合集）全部去除空行
    y = np.array(df['label'])           #将y数组化
    
    print('Data Preprocessed!')
    
    return X, y, X_lately               #返回数组化的X、y、X_lately

X, y, X_lately = general_data(df, 7)


#7.线性回归计算-----------------------------------------------------------------------------------------------------------

from sklearn import model_selection, svm
from sklearn.linear_model import LinearRegression

def linear_regression(X, y, X_lately):    #定义线性回归计算函数
    
    accuracy = 0                          #定义accuracy变量，即使出错，accuracy也有所显示
    forcast_set = [0]                     #定义forcast_set，同理
    
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.2) #进行train组跟test组的划分
    
    clf = LinearRegression(n_jobs = -1)  #定义clf为对全部数据进行线性回归计算
    clf.fit(X_train, y_train)            #计算对象为X_train组跟y_train组，得出拟合函数
    
    accuracy = clf.score(X_test, y_test) #计算clf线性回归的准确度
    forcast_set = clf.predict(X_lately)  #通过X_lately进行预测
    
    print('Linear Regression Calculated!')
    
    return accuracy, forcast_set         #返回准确率与预测值

accuracy, forcast_set = linear_regression(X, y, X_lately)


#9.写入数据库---------------------------------------------------------------------------------------------

def write_in():
    
    conn, cur = connect_db()
    
    sql = "insert into predicted_value(date, region, accuracy, day_1, day_2, day_3, day_4, day_5, day_6, day_7) value('%s', '%s', %s, %s, %s, %s, %s, %s, %s, %s)"%('2020-02-26', 'GD', accuracy, forcast_set[0], forcast_set[1], forcast_set[2], forcast_set[3], forcast_set[4], forcast_set[5], forcast_set[6])
    cur.execute(sql)
    conn.commit()

    sql = "insert into copredict_corr(date, corr_X1, corr_X2) value('%s', %s, %s)"%('2020-02-26', corr['y'][1], corr['y'][2])
    cur.execute(sql)
    conn.commit()
    
    print('Data Saved in DataBase!')

#运行程序---------------------------------------------------------------------------------------------   

if __name__ == '__main__': 
    write_in()                         #从入口程序运行程序
   
    #create_table_predict()
    
    #create_table_corr()

Connection Succeeded!
Connection Succeeded!
Data Table Obtained!
Connection Succeeded!
Connection Succeeded!
Data Table Obtained!
Connection Succeeded!
Connection Succeeded!
Data Table Obtained!
Date Generated!


d:\python\python37\lib\site-packages\ipykernel_launcher.py:139: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\python\python37\lib\site-packages\ipykernel_launcher.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\python\python37\lib\site-packages\ipykernel_launcher.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Variables for Calcutaion Obtained!
Coefficients of Correlation obtained!
Data Preprocessed!
Linear Regression Calculated!
Connection Succeeded!
Data Saved in DataBase!
